<a href="https://colab.research.google.com/github/nikojlieto/antennann/blob/main/firstnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
#helps move data forwards in the function
import torch.nn.functional as F
import tensorflow as tf

In [ ]:
#model class inherits nn.Module
class Model(nn.Module):
  #input layer (antennastate, prxetc, phaserads)
  #hidden layer 1 (some neurons)
  #hidden layer 2 (some neurons)
  #output (x and y)
  def __init__(self, output_features = 1, input_features=3, l1 = 9, l2 = 10):
    #instantiate nn.module
    super().__init__()
    out_x = 0
    out_y = 0
    #fc = fully connected
    self.fc1 = nn.Linear(3, l1)
    self.fc2 = nn.Linear(l1, l2)
    self.out1 = nn.Linear(l2, output_features)
    self.out2 = nn.Linear(l2, output_features)

    #training data labels - mark categories with likelihood percents
    #argmax(label) = category with most likelihood
    #for no categories feed in x and y coordinates for each sample


  def forward(self, x):
    #relu = rectified linear unit
    #if output < 0, correct to zero
    #x = F.relu(self.fc1(x))
    x = self.fc1(x)
    x = self.fc2(x)
    x1 = self.out1(x)
    x2 = self.out2(x)
    return x1, x2

In [ ]:
#manual seed for randomization
torch.manual_seed(4)
#create an instance of model
model = Model()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
githuburl = 'https://github.com/nikojlieto/firstnn/raw/main/Autofilled%20Placement.csv'
githuburl2 = 'https://github.com/nikojlieto/antennann/raw/main/localization%20XLP%20no%20freeform%20or%20timestamp.csv'
#githuburl.encode('utf-8').strip()
my_df = pd.read_csv(githuburl)
test_df = pd.read_csv(githuburl2)


In [ ]:
#list(test_df.columns)

In [ ]:
#split for training - X = features, y = outcome
X = my_df[['antennastate', 'prxetc', 'phaserads']]
y = my_df[['x', 'y']]


In [ ]:
#conversion to numpy arrays
X = X.values
y = y.values


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state =4)

In [ ]:
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
X_train.shape

torch.Size([4685, 3])

In [ ]:
y_train = torch.FloatTensor(y_train)
y_test = torch.FloatTensor(y_test)
y_train.shape

torch.Size([4685, 2])

In [ ]:
#set criterion of model to measure the error
criterion = nn.L1Loss()
#choose optimizer, lr = learning rate
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
#train model time oogie
#epoch - one run through of data
epochs = 100
#keep track of losses too
losses = []
#this should be a tensor instead of a tuple
#why is a single value being returned?

for i in range(100):
  #go forwards please
  #get predicted results here
  y_pred = model(X_train)
  #reformat from tuple
  y_pred = torch.stack([y_pred[0], y_pred[1]], axis=1)
  y_pred = torch.squeeze(y_pred)
  #measure the loss
  loss = criterion(y_pred, y_train)
  #to keep track
  losses.append(loss.detach().numpy())
  #print every 10 epoch
  if i%10 == 0:
    print(f'Epoch: {i} and loss: {loss}')
  #back propagation
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()



Epoch: 0 and loss: 0.9334233403205872
Epoch: 10 and loss: 0.41278713941574097
Epoch: 20 and loss: 0.3223509192466736
Epoch: 30 and loss: 0.28405895829200745
Epoch: 40 and loss: 0.2581321895122528
Epoch: 50 and loss: 0.23147915303707123
Epoch: 60 and loss: 0.2045806497335434
Epoch: 70 and loss: 0.17416398227214813
Epoch: 80 and loss: 0.15470436215400696
Epoch: 90 and loss: 0.1395118534564972


In [ ]:
#saving model
torch.save(model.state_dict(), 'trainedfirstnn.pt')
#it can be loaded too

In [ ]:
#test data time
with torch.no_grad():
  #reformatting
  X_test = torch.stack([X_test[0], X_test[1]], axis=1)
  X_test = torch.squeeze(X_test)
  print(X_test.shape)
  #put in features get predictions
  #the shape here isn't right!!
  y_eval = model.forward(X_test)
  #find the error
  loss = criterion(y_eval, y_test)
  print(loss)

torch.Size([3, 2])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (3x2 and 3x9)

In [ ]:
#new data eventually
test_columns =test_df[["antennastate", "prx linear ()", "phase (radians)"]]
test_row = test_columns.iloc[27]
tensor_row = torch.tensor(test_row.values)


with torch.no_grad():
  print(model(tensor_row))

RuntimeError: mat1 and mat2 must have the same dtype, but got Double and Float